In [5]:
num_sample = 300
num_burn = 20
sample_size = 800

In [ ]:
from dgp import *

# 1. Simulate or load data
min_degree = 2
max_degree = 4
adj = get_graph(sample_size, min_degree, max_degree, seed=1)

tau = np.array([-1.0, 0.50, -0.50])       # shape (3,)
rho = np.array([[0,0.1,0.2],
                [0.1,0,0.1],
                [0.2,0.1,0]])      # shape (3, 3), with 0s on the diagonal
nu = np.array([0.1,0,0,0.1,0,0,0.1,0,0]).reshape(3,3)       # shape (3, 3)
gamma = np.array([-2.00,0.50,-0.05,0.20,0.05,0.25,-0.08,0.30])     # shape (8,)
beta = np.array([-0.30,-0.60,-0.20,-0.20,-0.05,-0.10,-0.01,0.40,0.01,0.20])     # shape (10,)

Y_chain, A_chain, L_chain = sample_network_chain(adj, tau, rho, nu, gamma, beta, R=num_sample,
    burnin_R=num_burn, seed=0)

Y_chain = Y_chain[::3]
A_chain = A_chain[::3]
L_chain = L_chain[::3]

100%|██████████| 320/320 [00:21<00:00, 15.20it/s]


In [2]:
from agcEffect import *

agc_effect(
    adj,
    tau, rho, nu, beta,
    treatment_allocation=0.7,
    R=50,
    burnin_R=5,
    seed=0
)

{'average': 0.28154999999999997,
 'direct_effect': -0.14622500000000013,
 'spillover_effect': -0.11177499999999996,
 'psi_1_gamma': 0.23609999999999995,
 'psi_0_gamma': 0.3823250000000001,
 'psi_zero': 0.49410000000000004}

In [3]:
dir, dir2 = [], []
for i in range(Y_chain.shape[0]):
    Y = Y_chain[i]
    A = A_chain[i]
    L = L_chain[i]
    dir.append(np.mean(Y[A==1]) - np.mean(Y[A==0]))
    dir2.append(np.mean(A))

np.mean(dir), np.mean(dir2)

(-0.14233322487420697, 0.1940625)

In [1]:
n_cpu = 10

In [5]:
from run_pll import *
from utils import run_pll

args = [{'Y_chain': Y_chain,
         'A_chain': A_chain,
         'L_chain': L_chain,
         'adj': adj,
         'i': i} for i in range(len(Y_chain))]

res_list_dr = run_pll(run_dr, args, processes=n_cpu)
res_list_array_dr = np.array(res_list_dr)

Multiprocessing <function run_dr at 0x7f461ca34b80> in 100 tasks, with 100 processes...
Multiprocessing finished.


In [6]:
from run_pll import column_names

ret_mean_dr = res_list_array_dr.mean(axis=0)
ret_std_dr = res_list_array_dr.std(axis=0)
for i in range(len(column_names)):
    print(f"{column_names[i]}: {ret_mean_dr[i]:.5f} ± {ret_std_dr[i]:.5f}")

average: 0.28881 ± 0.05940
direct_effect: -0.12843 ± 0.11251
spillover_effect: -0.10709 ± 0.08320
psi_0_gamma: 0.37877 ± 0.07566
psi_zero: 0.48586 ± 0.02938
psi_1_gamma: 0.25034 ± 0.07897


In [7]:
from utils import run_pll
from run_pll import run_autognet

args = [{'Y_chain': Y_chain,
         'A_chain': A_chain,
         'L_chain': L_chain,
         'adj': adj,
         'i': i} for i in range(len(Y_chain))]

res_list_ag = run_pll(run_autognet, args, processes=n_cpu)
res_list_array_ag = np.array(res_list_ag)

Multiprocessing <function run_autognet at 0x7f46074e3a60> in 100 tasks, with 100 processes...
Multiprocessing finished.


In [8]:
from run_pll import column_names

ret_mean_ag = res_list_array_ag.mean(axis=0)
ret_std_ag = res_list_array_ag.std(axis=0)
for i in range(len(column_names)):
    print(f"{column_names[i]}: {ret_mean_ag[i]:.5f} ± {ret_std_ag[i]:.5f}")

average: 0.29693 ± 0.04038
direct_effect: -0.12625 ± 0.03741
spillover_effect: -0.09588 ± 0.05685
psi_0_gamma: 0.38906 ± 0.04652
psi_zero: 0.48494 ± 0.02778
psi_1_gamma: 0.26281 ± 0.04180


In [ ]:
from run_pll import *
from utils import run_pll

args = [{'Y_chain': Y_chain,
         'A_chain': A_chain,
         'L_chain': L_chain,
         'adj': adj,
         'i': i} for i in range(len(Y_chain))]

res_list_dr = run_pll(run_dr_em, args, processes=n_cpu)
res_list_array_dr = np.array(res_list_dr)

from run_pll import column_names

ret_mean_dr = res_list_array_dr.mean(axis=0)
ret_std_dr = res_list_array_dr.std(axis=0)
for i in range(len(column_names)):
    print(f"{column_names[i]}: {ret_mean_dr[i]:.5f} ± {ret_std_dr[i]:.5f}")

In [9]:
from dgp import *

# 1. Simulate or load data
min_degree = 2
max_degree = 4
adj = get_graph(sample_size, min_degree, max_degree, seed=1)

tau = np.array([-1.0, 0.50, -0.50])       # shape (3,)
rho = np.array([[0,0.1,0.2],
                [0.1,0,0.1],
                [0.2,0.1,0]])      # shape (3, 3), with 0s on the diagonal
nu = np.array([0.1,0,0,0.1,0,0,0.1,0,0]).reshape(3,3)       # shape (3, 3)
gamma = np.array([-2.00,0.50,-0.05,0.20,0.05,0.25,-0.08,0.30])     # shape (8,)
beta = np.array([-0.30,-0.60,-0.20,-0.20,-0.05,-0.10,-0.01,0.40,0.01,0.20])     # shape (10,)

Y_chain, A_chain, L_chain = sample_network_chain2(adj, tau, rho, nu, gamma, beta, R=num_sample,
    burnin_R=num_burn, seed=0)

Y_chain = Y_chain[::3]
A_chain = A_chain[::3]
L_chain = L_chain[::3]

100%|██████████| 320/320 [00:23<00:00, 13.80it/s]


In [10]:
from agcEffect import *

agc_effect2(
    adj,
    tau, rho, nu, beta,
    treatment_allocation=0.7,
    R=50,
    burnin_R=5,
    seed=0
)

{'average': 0.285325,
 'direct_effect': -0.04884999999999995,
 'spillover_effect': -0.09772500000000006,
 'psi_1_gamma': 0.27375000000000005,
 'psi_0_gamma': 0.3226,
 'psi_zero': 0.42032500000000006}

In [11]:
dir, dir2 = [], []
for i in range(Y_chain.shape[0]):
    Y = Y_chain[i]
    A = A_chain[i]
    L = L_chain[i]
    dir.append(np.mean(Y[A==1]) - np.mean(Y[A==0]))
    dir2.append(np.mean(A))

np.mean(dir), np.mean(dir2)

(-0.09679307080818747, 0.1940625)

In [12]:
from run_pll import *
from utils import run_pll

args = [{'Y_chain': Y_chain,
         'A_chain': A_chain,
         'L_chain': L_chain,
         'adj': adj,
         'i': i} for i in range(len(Y_chain))]

res_list_dr = run_pll(run_dr, args, processes=n_cpu)
res_list_array_dr = np.array(res_list_dr)

Multiprocessing <function run_dr at 0x7f461ca34b80> in 100 tasks, with 100 processes...
Multiprocessing finished.


In [13]:
from run_pll import column_names

ret_mean_dr = res_list_array_dr.mean(axis=0)
ret_std_dr = res_list_array_dr.std(axis=0)
for i in range(len(column_names)):
    print(f"{column_names[i]}: {ret_mean_dr[i]:.5f} ± {ret_std_dr[i]:.5f}")

average: 0.26744 ± 0.06072
direct_effect: -0.04394 ± 0.10966
spillover_effect: -0.12778 ± 0.07188
psi_0_gamma: 0.29871 ± 0.06580
psi_zero: 0.42649 ± 0.02695
psi_1_gamma: 0.25478 ± 0.08270


In [14]:
from utils import run_pll
from run_pll import run_autognet

args = [{'Y_chain': Y_chain,
         'A_chain': A_chain,
         'L_chain': L_chain,
         'adj': adj,
         'i': i} for i in range(len(Y_chain))]

res_list_ag = run_pll(run_autognet, args, processes=n_cpu)
res_list_array_ag = np.array(res_list_ag)

Multiprocessing <function run_autognet at 0x7f46074e3a60> in 100 tasks, with 100 processes...
Multiprocessing finished.


In [15]:
from run_pll import column_names

ret_mean_ag = res_list_array_ag.mean(axis=0)
ret_std_ag = res_list_array_ag.std(axis=0)
for i in range(len(column_names)):
    print(f"{column_names[i]}: {ret_mean_ag[i]:.5f} ± {ret_std_ag[i]:.5f}")

average: 0.25356 ± 0.03809
direct_effect: -0.07906 ± 0.03550
spillover_effect: -0.10533 ± 0.05176
psi_0_gamma: 0.31282 ± 0.04077
psi_zero: 0.41815 ± 0.02438
psi_1_gamma: 0.23376 ± 0.04167


In [ ]:
from run_pll import *
from utils import run_pll

args = [{'Y_chain': Y_chain,
         'A_chain': A_chain,
         'L_chain': L_chain,
         'adj': adj,
         'i': i} for i in range(len(Y_chain))]

res_list_dr = run_pll(run_dr_em, args, processes=n_cpu)
res_list_array_dr = np.array(res_list_dr)

from run_pll import column_names

ret_mean_dr = res_list_array_dr.mean(axis=0)
ret_std_dr = res_list_array_dr.std(axis=0)
for i in range(len(column_names)):
    print(f"{column_names[i]}: {ret_mean_dr[i]:.5f} ± {ret_std_dr[i]:.5f}")

In [16]:
from dgp import *

# 1. Simulate or load data
min_degree = 5
max_degree = 7
adj = get_graph(sample_size, min_degree, max_degree, seed=1)

tau = np.array([-1.0, 0.50, -0.50])       # shape (3,)
rho = np.array([[0,0.1,0.2],
                [0.1,0,0.1],
                [0.2,0.1,0]])      # shape (3, 3), with 0s on the diagonal
nu = np.array([0.1,0,0,0.1,0,0,0.1,0,0]).reshape(3,3)       # shape (3, 3)
gamma = np.array([-2.00,0.50,-0.05,0.20,0.05,0.25,-0.08,0.30])     # shape (8,)
beta = np.array([-0.30,-0.60,-0.20,-0.20,-0.05,-0.10,-0.01,0.40,0.01,0.20])     # shape (10,)

Y_chain, A_chain, L_chain = sample_network_chain(adj, tau, rho, nu, gamma, beta, R=num_sample,
    burnin_R=num_burn, seed=0)

Y_chain = Y_chain[::3]
A_chain = A_chain[::3]
L_chain = L_chain[::3]

100%|██████████| 320/320 [00:21<00:00, 15.11it/s]


In [17]:
from agcEffect import *

agc_effect(
    adj,
    tau, rho, nu, beta,
    treatment_allocation=0.7,
    R=50,
    burnin_R=5,
    seed=0
)

{'average': 0.21325,
 'direct_effect': -0.13562499999999994,
 'spillover_effect': -0.2715750000000001,
 'psi_1_gamma': 0.17255000000000004,
 'psi_0_gamma': 0.308175,
 'psi_zero': 0.5797500000000001}

In [18]:
dir, dir2 = [], []
for i in range(Y_chain.shape[0]):
    Y = Y_chain[i]
    A = A_chain[i]
    L = L_chain[i]
    dir.append(np.mean(Y[A==1]) - np.mean(Y[A==0]))
    dir2.append(np.mean(A))

np.mean(dir), np.mean(dir2)

(-0.1492641451885985, 0.22795)

In [19]:
from run_pll import *
from utils import run_pll

args = [{'Y_chain': Y_chain,
         'A_chain': A_chain,
         'L_chain': L_chain,
         'adj': adj,
         'i': i} for i in range(len(Y_chain))]

res_list_dr = run_pll(run_dr, args, processes=n_cpu)
res_list_array_dr = np.array(res_list_dr)

Multiprocessing <function run_dr at 0x7f461ca34b80> in 100 tasks, with 100 processes...
Multiprocessing finished.


In [20]:
from run_pll import column_names

ret_mean_dr = res_list_array_dr.mean(axis=0)
ret_std_dr = res_list_array_dr.std(axis=0)
for i in range(len(column_names)):
    print(f"{column_names[i]}: {ret_mean_dr[i]:.5f} ± {ret_std_dr[i]:.5f}")

average: 0.25471 ± 0.05166
direct_effect: -0.10202 ± 0.06924
spillover_effect: -0.20557 ± 0.07685
psi_0_gamma: 0.32749 ± 0.06322
psi_zero: 0.53306 ± 0.04246
psi_1_gamma: 0.22547 ± 0.05860


In [21]:
from utils import run_pll
from run_pll import run_autognet

args = [{'Y_chain': Y_chain,
         'A_chain': A_chain,
         'L_chain': L_chain,
         'adj': adj,
         'i': i} for i in range(len(Y_chain))]

res_list_ag = run_pll(run_autognet, args, processes=n_cpu)
res_list_array_ag = np.array(res_list_ag)

Multiprocessing <function run_autognet at 0x7f46074e3a60> in 100 tasks, with 100 processes...
Multiprocessing finished.


In [22]:
from run_pll import column_names

ret_mean_ag = res_list_array_ag.mean(axis=0)
ret_std_ag = res_list_array_ag.std(axis=0)
for i in range(len(column_names)):
    print(f"{column_names[i]}: {ret_mean_ag[i]:.5f} ± {ret_std_ag[i]:.5f}")

average: 0.25605 ± 0.04572
direct_effect: -0.11442 ± 0.03628
spillover_effect: -0.20003 ± 0.06948
psi_0_gamma: 0.34035 ± 0.05226
psi_zero: 0.54038 ± 0.03772
psi_1_gamma: 0.22593 ± 0.04617


In [ ]:
from run_pll import *
from utils import run_pll

args = [{'Y_chain': Y_chain,
         'A_chain': A_chain,
         'L_chain': L_chain,
         'adj': adj,
         'i': i} for i in range(len(Y_chain))]

res_list_dr = run_pll(run_dr_em, args, processes=n_cpu)
res_list_array_dr = np.array(res_list_dr)

from run_pll import column_names

ret_mean_dr = res_list_array_dr.mean(axis=0)
ret_std_dr = res_list_array_dr.std(axis=0)
for i in range(len(column_names)):
    print(f"{column_names[i]}: {ret_mean_dr[i]:.5f} ± {ret_std_dr[i]:.5f}")

In [23]:
from dgp import *

# 1. Simulate or load data
min_degree = 5
max_degree = 7
adj = get_graph(sample_size, min_degree, max_degree, seed=1)

tau = np.array([-1.0, 0.50, -0.50])       # shape (3,)
rho = np.array([[0,0.1,0.2],
                [0.1,0,0.1],
                [0.2,0.1,0]])      # shape (3, 3), with 0s on the diagonal
nu = np.array([0.1,0,0,0.1,0,0,0.1,0,0]).reshape(3,3)       # shape (3, 3)
gamma = np.array([-2.00,0.50,-0.05,0.20,0.05,0.25,-0.08,0.30])     # shape (8,)
beta = np.array([-0.30,-0.60,-0.20,-0.20,-0.05,-0.10,-0.01,0.40,0.01,0.20])     # shape (10,)

Y_chain, A_chain, L_chain = sample_network_chain2(adj, tau, rho, nu, gamma, beta, R=num_sample,
    burnin_R=num_burn, seed=0)

Y_chain = Y_chain[::3]
A_chain = A_chain[::3]
L_chain = L_chain[::3]

100%|██████████| 320/320 [00:22<00:00, 14.47it/s]


In [24]:
from agcEffect import *

agc_effect2(
    adj,
    tau, rho, nu, beta,
    treatment_allocation=0.7,
    R=50,
    burnin_R=5,
    seed=0
)

{'average': 0.20085,
 'direct_effect': 0.023425,
 'spillover_effect': -0.23290000000000005,
 'psi_1_gamma': 0.21085,
 'psi_0_gamma': 0.187425,
 'psi_zero': 0.42032500000000006}

In [25]:
dir, dir2 = [], []
for i in range(Y_chain.shape[0]):
    Y = Y_chain[i]
    A = A_chain[i]
    L = L_chain[i]
    dir.append(np.mean(Y[A==1]) - np.mean(Y[A==0]))
    dir2.append(np.mean(A))

np.mean(dir), np.mean(dir2)

(-0.07094678917810066, 0.22795)

In [26]:
from run_pll import *
from utils import run_pll

args = [{'Y_chain': Y_chain,
         'A_chain': A_chain,
         'L_chain': L_chain,
         'adj': adj,
         'i': i} for i in range(len(Y_chain))]

res_list_dr = run_pll(run_dr, args, processes=n_cpu)
res_list_array_dr = np.array(res_list_dr)

Multiprocessing <function run_dr at 0x7f461ca34b80> in 100 tasks, with 100 processes...
Multiprocessing finished.


In [27]:
from run_pll import column_names

ret_mean_dr = res_list_array_dr.mean(axis=0)
ret_std_dr = res_list_array_dr.std(axis=0)
for i in range(len(column_names)):
    print(f"{column_names[i]}: {ret_mean_dr[i]:.5f} ± {ret_std_dr[i]:.5f}")

average: 0.16713 ± 0.04120
direct_effect: -0.00984 ± 0.06437
spillover_effect: -0.25202 ± 0.06887
psi_0_gamma: 0.17532 ± 0.05089
psi_zero: 0.42734 ± 0.04236
psi_1_gamma: 0.16548 ± 0.04925


In [28]:
from utils import run_pll
from run_pll import run_autognet

args = [{'Y_chain': Y_chain,
         'A_chain': A_chain,
         'L_chain': L_chain,
         'adj': adj,
         'i': i} for i in range(len(Y_chain))]

res_list_ag = run_pll(run_autognet, args, processes=n_cpu)
res_list_array_ag = np.array(res_list_ag)

Multiprocessing <function run_autognet at 0x7f46074e3a60> in 100 tasks, with 100 processes...
Multiprocessing finished.


In [29]:
from run_pll import column_names

ret_mean_ag = res_list_array_ag.mean(axis=0)
ret_std_ag = res_list_array_ag.std(axis=0)
for i in range(len(column_names)):
    print(f"{column_names[i]}: {ret_mean_ag[i]:.5f} ± {ret_std_ag[i]:.5f}")

average: 0.15687 ± 0.03117
direct_effect: -0.03497 ± 0.02544
spillover_effect: -0.22626 ± 0.06194
psi_0_gamma: 0.18456 ± 0.03594
psi_zero: 0.41081 ± 0.03738
psi_1_gamma: 0.14959 ± 0.03291


In [ ]:
from run_pll import *
from utils import run_pll

args = [{'Y_chain': Y_chain,
         'A_chain': A_chain,
         'L_chain': L_chain,
         'adj': adj,
         'i': i} for i in range(len(Y_chain))]

res_list_dr = run_pll(run_dr_em, args, processes=n_cpu)
res_list_array_dr = np.array(res_list_dr)

from run_pll import column_names

ret_mean_dr = res_list_array_dr.mean(axis=0)
ret_std_dr = res_list_array_dr.std(axis=0)
for i in range(len(column_names)):
    print(f"{column_names[i]}: {ret_mean_dr[i]:.5f} ± {ret_std_dr[i]:.5f}")

In [4]:
from dgp import *

# 1. Simulate or load data
min_degree = 8
max_degree = 10
adj = get_graph(sample_size, min_degree, max_degree, seed=1)

tau = np.array([-1.0, 0.50, -0.50])       # shape (3,)
rho = np.array([[0,0.1,0.2],
                [0.1,0,0.1],
                [0.2,0.1,0]])      # shape (3, 3), with 0s on the diagonal
nu = np.array([0.1,0,0,0.1,0,0,0.1,0,0]).reshape(3,3)       # shape (3, 3)
gamma = np.array([-2.00,0.50,-0.05,0.20,0.05,0.25,-0.08,0.30])     # shape (8,)
beta = np.array([-0.30,-0.60,-0.20,-0.20,-0.05,-0.10,-0.01,0.40,0.01,0.20])     # shape (10,)

Y_chain, A_chain, L_chain = sample_network_chain(adj, tau, rho, nu, gamma, beta, R=num_sample,
    burnin_R=num_burn, seed=0)

Y_chain = Y_chain[::3]
A_chain = A_chain[::3]
L_chain = L_chain[::3]

NameError: name 'sample_size' is not defined

In [31]:
from agcEffect import *

agc_effect(
    adj,
    tau, rho, nu, beta,
    treatment_allocation=0.7,
    R=50,
    burnin_R=5,
    seed=0
)

{'average': 0.147975,
 'direct_effect': -0.11012500000000001,
 'spillover_effect': -0.4613499999999998,
 'psi_1_gamma': 0.11622500000000001,
 'psi_0_gamma': 0.22635000000000002,
 'psi_zero': 0.6876999999999999}

In [45]:
dir, dir2 = [], []
for i in range(Y_chain.shape[0]):
    Y = Y_chain[i]
    A = A_chain[i]
    L = L_chain[i]
    dir.append(np.mean(Y[A==1]) - np.mean(Y[A==0]))
    dir2.append(np.mean(A))

np.mean(dir), np.mean(dir2)

(-0.16009213116731968, 0.28511249999999994)

In [46]:
from run_pll import *
from utils import run_pll

args = [{'Y_chain': Y_chain,
         'A_chain': A_chain,
         'L_chain': L_chain,
         'adj': adj,
         'i': i} for i in range(len(Y_chain))]

res_list_dr = run_pll(run_dr, args, processes=n_cpu)
res_list_array_dr = np.array(res_list_dr)

Multiprocessing <function run_dr at 0x7f461ca34b80> in 100 tasks, with 100 processes...
Multiprocessing finished.


In [47]:
from run_pll import column_names

ret_mean_dr = res_list_array_dr.mean(axis=0)
ret_std_dr = res_list_array_dr.std(axis=0)
for i in range(len(column_names)):
    print(f"{column_names[i]}: {ret_mean_dr[i]:.5f} ± {ret_std_dr[i]:.5f}")

average: 0.21263 ± 0.03232
direct_effect: -0.10111 ± 0.03613
spillover_effect: -0.28673 ± 0.10653
psi_0_gamma: 0.28404 ± 0.04132
psi_zero: 0.57077 ± 0.09672
psi_1_gamma: 0.18293 ± 0.03210


In [48]:
from utils import run_pll
from run_pll import run_autognet

args = [{'Y_chain': Y_chain,
         'A_chain': A_chain,
         'L_chain': L_chain,
         'adj': adj,
         'i': i} for i in range(len(Y_chain))]

res_list_ag = run_pll(run_autognet, args, processes=n_cpu)
res_list_array_ag = np.array(res_list_ag)

Multiprocessing <function run_autognet at 0x7f46074e3a60> in 100 tasks, with 100 processes...
Multiprocessing finished.


In [49]:
from run_pll import column_names

ret_mean_ag = res_list_array_ag.mean(axis=0)
ret_std_ag = res_list_array_ag.std(axis=0)
for i in range(len(column_names)):
    print(f"{column_names[i]}: {ret_mean_ag[i]:.5f} ± {ret_std_ag[i]:.5f}")

average: 0.20445 ± 0.03912
direct_effect: -0.10174 ± 0.03410
spillover_effect: -0.29227 ± 0.08423
psi_0_gamma: 0.27999 ± 0.04868
psi_zero: 0.57226 ± 0.05464
psi_1_gamma: 0.17825 ± 0.03881


In [ ]:
from run_pll import *
from utils import run_pll

args = [{'Y_chain': Y_chain,
         'A_chain': A_chain,
         'L_chain': L_chain,
         'adj': adj,
         'i': i} for i in range(len(Y_chain))]

res_list_dr = run_pll(run_dr_em, args, processes=n_cpu)
res_list_array_dr = np.array(res_list_dr)

from run_pll import column_names

ret_mean_dr = res_list_array_dr.mean(axis=0)
ret_std_dr = res_list_array_dr.std(axis=0)
for i in range(len(column_names)):
    print(f"{column_names[i]}: {ret_mean_dr[i]:.5f} ± {ret_std_dr[i]:.5f}")

In [50]:
from dgp import *

# 1. Simulate or load data
min_degree = 8
max_degree = 10
adj = get_graph(sample_size, min_degree, max_degree, seed=1)

tau = np.array([-1.0, 0.50, -0.50])       # shape (3,)
rho = np.array([[0,0.1,0.2],
                [0.1,0,0.1],
                [0.2,0.1,0]])      # shape (3, 3), with 0s on the diagonal
nu = np.array([0.1,0,0,0.1,0,0,0.1,0,0]).reshape(3,3)       # shape (3, 3)
gamma = np.array([-2.00,0.50,-0.05,0.20,0.05,0.25,-0.08,0.30])     # shape (8,)
beta = np.array([-0.30,-0.60,-0.20,-0.20,-0.05,-0.10,-0.01,0.40,0.01,0.20])     # shape (10,)

Y_chain, A_chain, L_chain = sample_network_chain2(adj, tau, rho, nu, gamma, beta, R=num_sample,
    burnin_R=num_burn, seed=0)

Y_chain = Y_chain[::3]
A_chain = A_chain[::3]
L_chain = L_chain[::3]

100%|██████████| 320/320 [00:23<00:00, 13.91it/s]


In [51]:
from agcEffect import *

agc_effect2(
    adj,
    tau, rho, nu, beta,
    treatment_allocation=0.7,
    R=50,
    burnin_R=5,
    seed=0
)

{'average': 0.11192500000000001,
 'direct_effect': 0.04277499999999998,
 'spillover_effect': -0.33780000000000004,
 'psi_1_gamma': 0.1253,
 'psi_0_gamma': 0.08252500000000002,
 'psi_zero': 0.42032500000000006}

In [52]:
dir, dir2 = [], []
for i in range(Y_chain.shape[0]):
    Y = Y_chain[i]
    A = A_chain[i]
    L = L_chain[i]
    dir.append(np.mean(Y[A==1]) - np.mean(Y[A==0]))
    dir2.append(np.mean(A))

np.mean(dir), np.mean(dir2)

(-0.038306885948007297, 0.28511249999999994)

In [53]:
from run_pll import *
from utils import run_pll

args = [{'Y_chain': Y_chain,
         'A_chain': A_chain,
         'L_chain': L_chain,
         'adj': adj,
         'i': i} for i in range(len(Y_chain))]

res_list_dr = run_pll(run_dr, args, processes=n_cpu)
res_list_array_dr = np.array(res_list_dr)

Multiprocessing <function run_dr at 0x7f461ca34b80> in 100 tasks, with 100 processes...
Multiprocessing finished.


In [54]:
from run_pll import column_names

ret_mean_dr = res_list_array_dr.mean(axis=0)
ret_std_dr = res_list_array_dr.std(axis=0)
for i in range(len(column_names)):
    print(f"{column_names[i]}: {ret_mean_dr[i]:.5f} ± {ret_std_dr[i]:.5f}")

average: 0.08530 ± 0.02064
direct_effect: 0.00223 ± 0.01908
spillover_effect: -0.34108 ± 0.10829
psi_0_gamma: 0.08421 ± 0.02053
psi_zero: 0.42530 ± 0.09931
psi_1_gamma: 0.08645 ± 0.02269


In [55]:
from utils import run_pll
from run_pll import run_autognet

args = [{'Y_chain': Y_chain,
         'A_chain': A_chain,
         'L_chain': L_chain,
         'adj': adj,
         'i': i} for i in range(len(Y_chain))]

res_list_ag = run_pll(run_autognet, args, processes=n_cpu)
res_list_array_ag = np.array(res_list_ag)

Multiprocessing <function run_autognet at 0x7f46074e3a60> in 100 tasks, with 100 processes...
Multiprocessing finished.


In [56]:
from run_pll import column_names

ret_mean_ag = res_list_array_ag.mean(axis=0)
ret_std_ag = res_list_array_ag.std(axis=0)
for i in range(len(column_names)):
    print(f"{column_names[i]}: {ret_mean_ag[i]:.5f} ± {ret_std_ag[i]:.5f}")

average: 0.08250 ± 0.01997
direct_effect: -0.00429 ± 0.01441
spillover_effect: -0.30343 ± 0.07018
psi_0_gamma: 0.08660 ± 0.02274
psi_zero: 0.39003 ± 0.05660
psi_1_gamma: 0.08231 ± 0.02111


In [ ]:
from run_pll import *
from utils import run_pll

args = [{'Y_chain': Y_chain,
         'A_chain': A_chain,
         'L_chain': L_chain,
         'adj': adj,
         'i': i} for i in range(len(Y_chain))]

res_list_dr = run_pll(run_dr_em, args, processes=n_cpu)
res_list_array_dr = np.array(res_list_dr)

from run_pll import column_names

ret_mean_dr = res_list_array_dr.mean(axis=0)
ret_std_dr = res_list_array_dr.std(axis=0)
for i in range(len(column_names)):
    print(f"{column_names[i]}: {ret_mean_dr[i]:.5f} ± {ret_std_dr[i]:.5f}")